## Existing Agent Executor

In [2]:
from langchain.chat_models import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain_community.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.runnables import RunnablePassthrough
from permchain.langgraph import Graph, END

tools = [TavilySearchResults(max_results=1)]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# Choose the LLM that will drive the agent
llm = ChatOpenAI(model="gpt-3.5-turbo-1106")

# Construct the OpenAI Functions agent
agent_runnable = create_openai_functions_agent(llm, tools, prompt)

from langchain_core.agents import AgentFinish
# Define decision-making logic
def should_continue(data):
    # Logic to decide whether to continue in the loop or exit
    if isinstance(data['agent_outcome'], AgentFinish):
        return "exit"
    else:
        return "continue"
    
def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    observation = {t.name: t for t in tools}[agent_action.tool].invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data
    
    

# Define agents
agent = RunnablePassthrough.assign(
    agent_outcome = agent_runnable
)


# Define a new graph
workflow = Graph()

workflow.add_node("agent", agent)
workflow.add_node("tools", execute_tools)

workflow.set_entry_point("agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "exit": END
    }
)

workflow.add_edge('tools', 'agent')

chain = workflow.compile()

/Users/harrisonchase/workplace/langchain/libs/core/langchain_core/_api/beta_decorator.py:160: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
/Users/harrisonchase/workplace/langchain/libs/core/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [3]:
chain.invoke({"input": "what is the weather in sf", "intermediate_steps": []})

{'input': 'what is the weather in sf',
 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query":"weather in San Francisco"}'}})]),
   [{'url': 'https://www.oddsshopper.com/articles/nfl/los-angeles-rams-san-francisco-49ers-weather-report-01-07-2024',
     'content': "find a current San Francisco-Los Angeles weather report.  Weather Forecasted for the Rams-49ers Football Game  November 9, 2023 7:58 am 7:58 am  Rams-49ers Weather ReportJanuary 07, 2024 | 2:03 AM by OddsShopper Staff. This Week 18 NFL game is to be played in Santa Clara, CA at Levi's Stadium. Below you can find a current San Francisco-Los Angeles weather report. 🏈 Game time: 4:25PM EST: 📅 Game date: Sunday, January 7th, 2

In [4]:
for s in chain.stream({"input": "what is the weather in sf", "intermediate_steps": []}, output=["agent", "tools"]):
    print(s)

{'tools': {'input': 'what is the weather in sf', 'intermediate_steps': [], 'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query":"weather in San Francisco"}'}})])}}
{'agent': {'input': 'what is the weather in sf', 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input={'query': 'weather in San Francisco'}, log="\nInvoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`\n\n\n", message_log=[AIMessage(content='', additional_kwargs={'function_call': {'name': 'tavily_search_results_json', 'arguments': '{"query":"weather in San Francisco"}'}})]), [{'url': 'https://www.whereandwhen.net/when/north-america/california/san-francisco-ca/

### Make it always call a specific tool first

Let's imagine that we ALWAYS want to call a specific tool to start. We can do that easily by modifying the graph

In [11]:
from langchain_core.agents import AgentActionMessageLog

def first_agent(inputs):
    inputs["agent_outcome"] = AgentActionMessageLog(tool="tavily_search_results_json", tool_input=inputs["input"], log="", message_log=[])
    return inputs

In [12]:
workflow = Graph()

workflow.add_node("first_agent", first_agent)
workflow.add_node("agent", agent)
workflow.add_node("tools", execute_tools)

workflow.set_entry_point("first_agent")

workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "continue": "tools",
        "exit": END
    }
)
workflow.add_edge('first_agent', 'tools')
workflow.add_edge('tools', 'agent')

chain = workflow.compile()

In [13]:
for s in chain.stream({"input": "what is the weather in sf", "intermediate_steps": []}, output=["agent", "tools", "first_agent"]):
    print(s)

{'tools': {'input': 'what is the weather in sf', 'intermediate_steps': [], 'agent_outcome': AgentActionMessageLog(tool='tavily_search_results_json', tool_input='what is the weather in sf', log='', message_log=[])}}
{'agent': {'input': 'what is the weather in sf', 'intermediate_steps': [(AgentActionMessageLog(tool='tavily_search_results_json', tool_input='what is the weather in sf', log='', message_log=[]), [{'url': 'https://www.whereandwhen.net/when/north-america/california/san-francisco-ca/january/', 'content': 'Best time to go to San Francisco? Weather in San Francisco in january 2024  How was the weather last january? Here is the day by day recorded weather in San Francisco in january 2023:  Seasonal average climate and temperature of San Francisco in january  8% 46% 29% 12% 8% Evolution of daily average temperature and precipitation in San Francisco in januaryWeather in San Francisco in january 2024. The weather in San Francisco in january comes from statistical datas on the past y

## Reflexion Agent

In [5]:
from langchain.agents import AgentExecutor, BaseMultiActionAgent, Tool
from langchain.schema import AgentAction, AgentFinish
from langchain_core.language_models.chat_models import BaseChatModel
from langchain.chains import LLMChain

from langchain.globals import set_llm_cache

from dotenv import load_dotenv

from pydantic import BaseModel

from langchain.chat_models import ChatOpenAI
from langchain.cache import SQLiteCache

from langchain_core.output_parsers import BaseOutputParser

from langchain.prompts.chat import ChatPromptTemplate
from langchain.callbacks import get_openai_callback
from langchain.tools.tavily_search import TavilySearchResults
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain.pydantic_v1 import BaseModel
import os

from langchain.agents import AgentType, initialize_agent, load_tools

set_llm_cache(SQLiteCache(database_path=".langchain.db"))


llm = ChatOpenAI(
    temperature=0.0,
    max_tokens=2000,
    max_retries=100,
    model="gpt-4-1106-preview",
)

search = TavilySearchAPIWrapper()
tavily_tool = TavilySearchResults(api_wrapper=search, max_results=5)

NEXT_STEP_TEMPLATE = """You are expert researcher trying answer a question ~250 words. You are asked to answer the following question: {question}

The way you are going to answer the question is as follows:

1. Revise your previous answer using the new information.
    - You should use the previous critique to add important information to your answer.
        _ You MUST include numerical citations in your revised answer to ensure it can be verified.
        - Add a "References" section to the bottom of your answer (which does not count towards the word limit). In form of:
            - [1] https://example.com
            - [2] https://example.com
    - You should use the previous critique to remove superfluous information from your answer and make SURE it is not more than 250 words.
2. Reflect and critique your answer. Specifically, you should:
    - Think about what is missing from your answer.
    - Think about what is superfluous in your answer.
    - Think about what search query you should use next to improve your answer.
  Give your answer in exactly 2 parts. The first should address what is missing from your answer. The second should address what could be removed from your answer. Your should be VERY harsh as we really want to improve the answer.
3. Give the search query you came up with to improve your answer.

Previous steps: 

{previous_steps}

===

Format your answer as follows:

Revised answer: [give your revised answer based on the previous critique and new information from the search engine then the "References" section]
Critique: [give your harsh critique of your revised answer in 2 parts: what is missing and what is superfluous]
Search query: [give the new search query you came up with to enter into the search engine to improve your answer. If you have more than one, make sure they are comma separated and in quotes]

SAY NOTHING else please."""

INITIAL_ANSWER_TEMPLATE = """You are expert researcher trying answer a question ~250 words. You are asked to answer the following question: {question}

The way you are going to answer the question is as follows:

1. Give a detailed in ~250 words.
2. Reflect and critique your answer. Specifically, you should:
    - Think about what is missing from your answer.
    - Think about what is superfluous in your answer.
    - Think about what search query you should use next to improve your answer.
  Give your answer in exactly 2 parts. The first should address what is missing from your answer. The second should address what could be removed from your answer. Your should be VERY harsh as we really want to improve the answer.
3. Give the search query you came up with to improve your answer.

===

Format your answer as follows:

Answer: [give your initial answer]
Critique: [give your harsh critique of your answer in 2 parts: what is missing and what is superfluous]
Search query: [give the search query you came up with to improve your answer. If you have more than one, make sure they are comma separated and in quotes]

SAY NOTHING else please."""


class ReflexionStep(BaseModel):
    """A single step in the reflexion process."""

    answer: str
    critique: str
    search_query: str

    def __str__(self):
        return f"Answer: {self.answer}\nCritique: {self.critique}\nSearch query: {self.search_query}"

def _parse_reflexion_step(output: str) -> tuple[str, str, str]:
    # find answer using .split()
    if ("Answer:" not in output and "Revised answer:" not in output) or not "Critique:" in output or not "Search query:" in output:
        raise ValueError(f"The output is not formatted correctly. Output: {output}")
    if "Answer:" in output:
        answer = output.split("Answer:")[1].split("Critique:")[0].strip()
    else:
        answer = output.split("Revised answer:")[1].split("Critique:")[0].strip()
    critique = output.split("Critique:")[1].split("Search query:")[0].strip()
    search_query = output.split("Search query:")[1].strip()
    return answer, critique, search_query

class ReflexionStepParser(BaseOutputParser[ReflexionStep]):
    """Parser for the reflexion step."""

    def parse(self, output: str) -> ReflexionStep:
        """Parse the output."""
        # try to find answer or initial answer
        answer, critique, search_query = _parse_reflexion_step(output)
        return ReflexionStep(
            answer=answer, critique=critique, search_query=search_query
        )

/Users/harrisonchase/workplace/langchain/libs/core/langchain_core/_api/deprecation.py:189: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(


In [6]:
initial_chain = RunnablePassthrough.assign(
    agent_outcome = ChatPromptTemplate.from_template(INITIAL_ANSWER_TEMPLATE) | llm | ReflexionStepParser() | (lambda x: AgentAction(
                    tool="tavily_search_results_json",
                    tool_input=x.search_query,
                    log=str(x),
                ))
)

def prep_next(inputs):
    intermediate_steps = inputs["intermediate_steps"]
    previous_steps = list[str]()

    for i, (action, observation) in enumerate(intermediate_steps, start=1):
        last_step_str = f"""Step {i}:

{action.log}

Search output for "{action.tool_input}":

{observation}"""
        previous_steps.append(last_step_str)

    previous_steps_str = "\n\n".join(previous_steps)
    inputs["previous_steps"] = previous_steps_str
    return inputs
    
next_chain = RunnablePassthrough.assign(
    agent_outcome = prep_next | ChatPromptTemplate.from_template(NEXT_STEP_TEMPLATE) | llm | ReflexionStepParser() | (lambda x: AgentAction(
                tool="tavily_search_results_json",
                tool_input=x.search_query,
                log=str(x),
            ))
)

def finish(inputs):
    intermediate_steps = inputs["intermediate_steps"]
    last_action, _ = intermediate_steps[-1]
    last_step_str = last_action.log
    # extract answer
    answer, _, _ = _parse_reflexion_step(last_step_str)

    first_action, _ = intermediate_steps[0]
    first_step_str = first_action.log
    # extract answer
    initial_answer, _, _ = _parse_reflexion_step(first_step_str)

    return AgentFinish(
        log="Reached max steps.",
        return_values={"output": answer, "initial_answer": initial_answer},
    )


def execute_tools(data):
    agent_action = data.pop('agent_outcome')
    observation = {t.name: t for t in tools}[agent_action.tool].invoke(agent_action.tool_input)
    data['intermediate_steps'].append((agent_action, observation))
    return data


In [7]:
workflow = Graph()

# add actors
workflow.add_node("initial", initial_chain)
workflow.add_node("next", next_chain)
workflow.add_node("finish", finish)
workflow.add_node("tools", execute_tools)

# Enter with initial actor, then loop through tools -> next steps until finished
workflow.set_entry_point('initial')

workflow.add_edge('initial', 'tools')
workflow.add_conditional_edges(
    'tools',
    lambda x: "exit" if len(x['intermediate_steps']) >= 2 else "continue",
    {
        "continue": 'next',
        "exit": 'finish'
    }
)
workflow.add_edge('next', 'tools')
workflow.set_finish_point('finish')

chain = workflow.compile()

chain.invoke({"question": "what is the weather in sf", "intermediate_steps": []})

AgentFinish(return_values={'output': "The current weather in San Francisco is experiencing colder than usual temperatures with a frost advisory and freeze warning in effect for the morning of January 7, 2024. The city's low is forecasted to remain around 41 degrees Fahrenheit [1]. This information is crucial for residents and visitors to take appropriate precautions against the cold. San Francisco's weather is generally mild with microclimates, but such advisories indicate that temperatures can drop significantly, especially in winter months. It is important to stay updated with the latest weather reports as conditions can change rapidly.\n\nReferences:\n[1] https://www.mercurynews.com/2024/01/07/bay-area-and-beyond-frost-advisory-and-freeze-warning-monday-morning/", 'initial_answer': "The weather in San Francisco (SF) is characterized by a mild, Mediterranean-like climate with wet winters and dry summers. The city's unique topography and coastal location result in microclimates, where